In [42]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
seed=42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# Load data
from src.data import make_dataset
molecules=make_dataset.load()

In [44]:
# Build the features
from src.features import build_features
X=build_features.build(molecules)

[<rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a2195cdf0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a219512b0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a21951940>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a21951490>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a21951e90>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a21951ee0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a219e6df0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a219e67b0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a219e6170>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a2198b670>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a2198b490>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a1ddc12b0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x1a219f4a80>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect ob

In [48]:
# Build the targets
from src.features import build_targets
y=build_targets.build_hit(molecules)


Processed 33 hits.


In [53]:
# Create train/test sets
from src.models import split_data
X_train, X_test, y_train, y_test, ind_train, ind_test = split_data.split(0.4,X,y,seed)
mol_test=[molecules[x] for x in ind_test]
mol_train=[molecules[x] for x in ind_train]
    

Fraction hits: train: 0.030, test: 0.036


In [51]:
# Train model
from src.models import train_model
models=[train_model.train_lr(X_train,y_train,seed),train_model.train_rfc(X_train,y_train,seed,n_estimators=10)]

/opt/anaconda3/envs/moldist2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/moldist2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/moldist2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/moldist2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/moldist2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default so

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
Cross-validation score = 0.804 += 0.170
RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)
Cross-validation score = 0.813 += 0.162


In [58]:
# Test model
from src.models import predict_model
for model in models:
    print("------")
    print(model)
    print("Train:")
    yp_train=predict_model.predict(model,X_train,y_train,mol_train)
    print("Test:")
    yp_test=predict_model.predict(model,X_test,y_test,mol_test)


------
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
Train:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       559
         1.0       1.00      1.00      1.00        17

    accuracy                           1.00       576
   macro avg       1.00      1.00      1.00       576
weighted avg       1.00      1.00      1.00       576

[[559   0]
 [  0  17]]
ROC_AUC Score = 1.000
Test:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99       370
         1.0       0.67      0.57      0.62        14

    accuracy                           0.97       384
   macro avg       0.83      0.78      0.80       384
weighte